In [1]:
# !pip install keras-tuner tensorflow gensim numpy pandas --quiet --upgrade

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import pickle
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Input, Embedding, Conv1D, MaxPool1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras_tuner as kt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [4]:
EPOCHS=500
BATCH_SIZE=128
PATIENCE=5

In [5]:
# base_path = '/content/drive/MyDrive/fake_news/'
base_path = ''

# Load Embeddings & Inputs

In [6]:
with open(base_path+'artifacts/embeddings_inputs.pkl', 'rb') as f:
    loaded_input_items = pickle.load(f)

In [7]:
embedding_matrix = loaded_input_items['embedding_matrix']
X_train_pad = loaded_input_items['X_train_pad']
X_val_pad = loaded_input_items['X_val_pad']
X_test_pad = loaded_input_items['X_test_pad']
y_train = loaded_input_items['y_train']
y_val = loaded_input_items['y_val']
y_test = loaded_input_items['y_test']

In [8]:
VOCAB_SIZE = embedding_matrix.shape[0]
EMBEDDING_DIM = embedding_matrix.shape[1]
MAX_LEN = len(X_train_pad[0])

In [9]:
print(f"Embedding dimension: {EMBEDDING_DIM}\nVocab size: {VOCAB_SIZE}\nMaximum input length: {MAX_LEN}")

Embedding dimension: 500
Vocab size: 35756
Maximum input length: 588


# Tuning CNN - 1 Convolution layer

In [14]:
def build_model1(hp):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=hp.Choice('embeddings_trainable', values=[True, False])))
    
    model.add(Conv1D(filters=hp.Choice(f'conv1_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv1_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(units=hp.Choice(f'dense_units', values=[16, 32, 64, 128]),
                        activation='relu'))
    model.add(Dropout(hp.Float(f'dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05, 0.1])),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [15]:
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=EPOCHS,
#     factor=3,
#     directory='tuner_results',
#     project_name='cnn_hyperband')
tuner = kt.RandomSearch(
    build_model1,
    objective='val_accuracy',
    max_trials=33,
    executions_per_trial=1,
    directory=base_path+'tuner_results',
    project_name='cnn_randomSearch')

In [16]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
tuner.search(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Trial 17 Complete [00h 02m 41s]
val_accuracy: 0.9479926824569702

Best val_accuracy So Far: 0.9656326174736023
Total elapsed time: 01h 23m 09s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
0                 |1                 |embeddings_trainable
256               |512               |conv1_filters
4                 |4                 |conv1_kernel_size
64                |16                |dense_units
0.3               |0.3               |dense_dropout
0.05              |0.001             |learning_rate

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 37s 349ms/step - accuracy: 0.6284 - loss: 1.0936 - val_accuracy: 0.7865 - val_loss: 0.4599
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 36s 349ms/step - accuracy: 0.8410 - loss: 0.4018 - val_accuracy: 0.8461 - val_loss: 0.3539
Epoch 3/500
  4/103 ━━━━━━━━━━━━━━━━━━━━ 35s 359ms/step - accuracy: 0.8315 - loss: 0.4068

KeyboardInterrupt: 

In [ ]:
best_model1 = tuner.get_best_models(num_models=1)[0]
best_hp1 = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp1.values)

In [ ]:
best_loss1, best_accuracy1 = best_model1.evaluate(X_test_pad, y_test, verbose=0)
print("Best CNN Accuracy:", best_accuracy1)
print("Best CNN Loss:", best_loss1)

# Tuning CNN - 2 Convolution layers

In [ ]:
def build_model2(hp):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=hp.Choice('embeddings_trainable', values=[True, False])))
    
    model.add(Conv1D(filters=hp.Choice(f'conv1_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv1_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(filters=hp.Choice(f'conv2_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv2_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(units=hp.Choice(f'dense_units', values=[16, 32, 64, 128]),
                        activation='relu'))
    model.add(Dropout(hp.Float(f'dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05, 0.1])),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [ ]:
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=EPOCHS,
#     factor=3,
#     directory='tuner_results',
#     project_name='cnn_hyperband')
tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=33,
    executions_per_trial=1,
    directory=base_path+'tuner_results',
    project_name='cnn_randomSearch')

In [ ]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
tuner.search(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

In [ ]:
best_model2 = tuner.get_best_models(num_models=1)[0]
best_hp2 = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp2.values)

In [ ]:
best_loss2, best_accuracy2 = best_model2.evaluate(X_test_pad, y_test, verbose=0)
print("Best CNN Accuracy:", best_accuracy2)
print("Best CNN Loss:", best_loss2)

# Tuning CNN - 3 Convolution layers

In [ ]:
def build_model3(hp):
    model = Sequential()
    model.add(Input(shape=(MAX_LEN,)))
    model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=hp.Choice('embeddings_trainable', values=[True, False])))
    
    model.add(Conv1D(filters=hp.Choice(f'conv1_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv1_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(filters=hp.Choice(f'conv2_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv2_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))
    model.add(MaxPool1D(pool_size=2))

    model.add(Conv1D(filters=hp.Choice(f'conv3_filters', values=[64, 128, 256, 512]),
                         kernel_size=hp.Int(f'conv3_kernel_size', min_value=2, max_value=5, step=1),
                         strides=1, padding='valid', activation='relu'))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(units=hp.Choice(f'dense_units', values=[16, 32, 64, 128]),
                        activation='relu'))
    model.add(Dropout(hp.Float(f'dense_dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05, 0.1])),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

In [ ]:
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=EPOCHS,
#     factor=3,
#     directory='tuner_results',
#     project_name='cnn_hyperband')
tuner = kt.RandomSearch(
    build_model3,
    objective='val_accuracy',
    max_trials=33,
    executions_per_trial=1,
    directory=base_path+'tuner_results',
    project_name='cnn_randomSearch')

In [ ]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
tuner.search(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

In [ ]:
best_model3 = tuner.get_best_models(num_models=1)[0]
best_hp3 = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hp3.values)

In [ ]:
best_loss3, best_accuracy3 = best_model3.evaluate(X_test_pad, y_test, verbose=0)
print("Best CNN Accuracy:", best_accuracy3)
print("Best CNN Loss:", best_loss3)

In [ ]:
bests = {'models':[best_model1, best_model2, best_model3],
        'accuracies':[best_accuracy1, best_accuracy2, best_accuracy3]}
n = np.argmax(bests['accuracies'])
print(f"Best number of convulation layers: {n}")
best_model = bests['models'][n]

In [ ]:
best_model.save(base_path+'artifacts/tuned_cnn.keras')